In [20]:
from langchain.chat_models import ChatOpenAI


chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
    )

b = chat.predict("How many planets are there?", max_tokens=150)

b

'As of now, there are eight recognized planets in our solar system. They are:\n\n1. Mercury\n2. Venus\n3. Earth\n4. Mars\n5. Jupiter\n6. Saturn\n7. Uranus\n8. Neptune\n\nIn addition to these, there are also dwarf planets, such as Pluto, Eris, Haumea, and Makemake, which are not classified as full-fledged planets but are still significant celestial bodies. In total, there are thousands of exoplanets (planets outside our solar system) that have been discovered, but the exact number is constantly changing as new discoveries are made.'

In [21]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

In [23]:

messages = [
    SystemMessage(
        content="You are a geography expert. And you only reply in Koean.",
    ),
    AIMessage(content="안녕 나는 홍길동이야!"),
    HumanMessage(
        content="What is the distance between Seoul and Deajeon. Also, what is your name?",
    ),
]

chat.predict_messages(messages)

AIMessage(content='서울과 대전 사이의 거리는 약 140킬로미터입니다. 제 이름은 홍길동입니다!')

In [24]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate


template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}",
)

prompt = template.format(country_a="Korea", country_b="Japan")


chat.predict(prompt)

'The distance between Korea and Japan is approximately 1,000 kilometers (620 miles) across the Korea Strait.'

In [26]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "Ciao, mi chiamo {name}!"),
        (
            "human",
            "What is the distance between {country_a} and {country_b}. Also, what is your name?",
        ),
    ]
)

prompt = template.format_messages(
    language="Korean",
    name="김철수",
    country_a="Korea",
    country_b="England",
)


chat.predict_messages(prompt)


AIMessage(content='한국과 영국 사이의 거리는 약 8700km입니다. 제 이름은 김철수입니다.')

In [28]:
from langchain.schema import BaseOutputParser


class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

In [29]:
p = CommaOutputParser()
p.parse("Hello, how, are ,you")

['Hello', 'how', 'are', 'you']

In [30]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase.Do NOT reply with anything else."),
    ("human", "{question}")
])

In [31]:
prompt = template.format_messages(
    max_items=10,
    question="What are the planets"
)

In [32]:
chat.predict_messages(prompt)

AIMessage(content='mercury, venus, earth, mars, jupiter, saturn, uranus, neptune')

In [ ]:
result = chat.predict("What are the colors")
p.parse(result)

['Colors are visual perceptions that are created when light is reflected off an object and detected by the human eye. Some common colors include red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'and brown.']

In [39]:
prompt = template.format_messages(
    max_items=7,
    question="What are the colors in the rainbow"
)
result = chat.predict_messages(prompt)
p.parse(result.content)

['red', 'orange', 'yellow', 'green', 'blue', 'indigo', 'violet']

In [40]:
chain = template | chat | CommaOutputParser()

In [41]:
chain.invoke({"max_items": 5, "question": "What are the pokemons?"})

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']

In [58]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a world-class international chef.
            You create easy to follow recipes for any type of cuisine 
            with easy to find ingredients.
            """,
        ),
        ("human", "I want to cook {cuisine} food."),
    ]
)

chef_chain = chef_prompt | chat

In [59]:
result = chef_chain.invoke({"cuisine": "Italian"})
result

AIMessage(content="Great choice! Italian cuisine is delicious and full of flavor. How about trying a classic dish like Spaghetti Aglio e Olio? It's a simple yet flavorful pasta dish that is quick and easy to make. Here's the recipe:\n\nIngredients:\n- 8 oz spaghetti\n- 4 cloves of garlic, thinly sliced\n- 1/4 cup extra virgin olive oil\n- 1/2 tsp red pepper flakes\n- Salt and pepper to taste\n- Fresh parsley, chopped (for garnish)\n- Grated Parmesan cheese (optional)\n\nInstructions:\n1. Cook the spaghetti according to package instructions until al dente. Reserve 1 cup of pasta water before draining the spaghetti.\n2. In a large skillet, heat the olive oil over medium heat. Add the sliced garlic and red pepper flakes. Cook until the garlic is golden and fragrant, about 1-2 minutes. Be careful not to burn the garlic.\n3. Add the cooked spaghetti to the skillet, along with a splash of the reserved pasta water. Toss everything together to coat the spaghetti with the garlic-infused oil. Ad

In [60]:
veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a vegetarian chef specialized on making traditional
            recipies vegetarian. You find alternative ingredients and explain
            their preparation. You don't radically modify the recipe.
            If there is no alternative for a food just say you don't know how
            to replace it.
            """
        ),
        ("human", "{recipe}"),
    ]
)

veg_chain = veg_chef_prompt | chat

In [61]:
final_chain = {"recipe": chef_chain} | veg_chain

In [62]:
final_chain.invoke({"cuisine": "indian"})

AIMessage(content="I'd be happy to help you make a vegetarian version of Chicken Tikka Masala! Instead of chicken, we can use paneer, a type of Indian cottage cheese that is firm and holds its shape well when cooked. Here's how you can prepare the paneer as a substitute for the chicken:\n\n1. **Paneer**: \n   - Cut the paneer into cubes or slices, similar in size to chicken pieces.\n   - Marinate the paneer in a mixture of yogurt, ginger-garlic paste, turmeric, cumin, coriander, garam masala, and a little lemon juice. Let it marinate for at least 30 minutes to allow the flavors to infuse.\n\nOnce your paneer is marinated, you can follow a traditional Chicken Tikka Masala recipe, replacing the chicken with the marinated paneer. Enjoy your vegetarian version of this classic Indian dish!")

In [56]:
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a world-class international chef.
            You create easy to follow recipes for any type of cuisine 
            with easy to find ingredients.
            """,
        ),
        ("human", "I want to cook {cuisine} food."),
    ]
)

chef_chain = chef_prompt | chat

veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a vegetarian chef specialized on making traditional
            recipies vegetarian. You find alternative ingredients and explain
            their preparation. You don't radically modify the recipe.
            If there is no alternative for a food just say you don't know how
            to replace it.
            """
        ),
        ("human", "{recipe}"),
    ]
)

veg_chain = veg_chef_prompt | chat
final_chain = {"recipe": chef_chain} | veg_chain

In [57]:
final_chain.invoke({"cuisine": "indian"})

Great choice! Indian cuisine is full of delicious flavors and spices. How about trying to make a classic dish like Chicken Tikka Masala? It's a popular Indian dish that is loved by many. Here's a simple recipe for you to try at home:

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 teaspoons ground cumin
- 2 teaspoons paprika
- 1 teaspoon ground cinnamon
- 1 teaspoon ground turmeric
- 1 teaspoon ground coriander
- 1 teaspoon cayenne pepper (adjust to taste)
- Salt and pepper to taste
- 2 tablespoons vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1 tablespoon grated ginger
- 1 can (14 oz) tomato sauce
- 1 cup heavy cream
- Fresh cilantro, chopped (for garnish)
- Cooked rice or naan bread (for serving)

Instructions:
1. In a bowl, mix together the yogurt, lemon juice, cumin, paprika, cinnamon, turmeric, coriander, cayenne pepper, salt, and pepper. Add the chicken pieces and coat t

AIMessageChunk(content="To make a vegetarian version of Chicken Tikka Masala, you can replace the chicken with a plant-based alternative such as tofu or paneer. Here's how you can adapt the recipe:\n\nIngredients:\n- 1 lb firm tofu or paneer, cut into bite-sized pieces\n- 1 cup plain yogurt (you can use dairy-free yogurt for a vegan version)\n- 2 tablespoons lemon juice\n- 2 teaspoons ground cumin\n- 2 teaspoons paprika\n- 1 teaspoon ground cinnamon\n- 1 teaspoon ground turmeric\n- 1 teaspoon ground coriander\n- 1 teaspoon cayenne pepper (adjust to taste)\n- Salt and pepper to taste\n- 2 tablespoons vegetable oil\n- 1 onion, finely chopped\n- 3 cloves garlic, minced\n- 1 tablespoon grated ginger\n- 1 can (14 oz) tomato sauce\n- 1 cup coconut cream (or another dairy-free alternative)\n- Fresh cilantro, chopped (for garnish)\n- Cooked rice or naan bread (for serving)\n\nInstructions:\n1. Follow the same marinating process as the original recipe, but use tofu or paneer instead of chicken.

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.7,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# Define the Code Implementation Chain
poetry_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a poet specialized in writing poetry about programming languages. 
            Your task is to create a creative and insightful poem that captures the essence, features, 
            or philosophy of the given programming language.
            """,
        ),
        ("human", "Write a poem about the programming language {language}.")
    ]
)

poetry_chain = poetry_prompt | chat
poetry_chain.invoke({"language": "Python"})

In the land of code, where serpents roam,
There lies a language, Python known.
Graceful and sleek, its syntax clear,
A language loved far and near.

Indented blocks like a dance routine,
Readability, its shining sheen.
Dynamic typing, flexibility's key,
In Python, freedom dances free.

From web development to AI's might,
Python shines in the day and night.
Libraries vast, like stars above,
Nurturing growth, empowering love.

Guido's creation, a gift profound,
Guiding devs the whole world round.
Beneath its surface, power untold,
In Python's embrace, stories unfold.

So, let us code with Python's grace,
In its realm, we find our place.
A language of beauty, logic, and art,
Python, the language that warms the heart.

AIMessageChunk(content="In the land of code, where serpents roam,\nThere lies a language, Python known.\nGraceful and sleek, its syntax clear,\nA language loved far and near.\n\nIndented blocks like a dance routine,\nReadability, its shining sheen.\nDynamic typing, flexibility's key,\nIn Python, freedom dances free.\n\nFrom web development to AI's might,\nPython shines in the day and night.\nLibraries vast, like stars above,\nNurturing growth, empowering love.\n\nGuido's creation, a gift profound,\nGuiding devs the whole world round.\nBeneath its surface, power untold,\nIn Python's embrace, stories unfold.\n\nSo, let us code with Python's grace,\nIn its realm, we find our place.\nA language of beauty, logic, and art,\nPython, the language that warms the heart.")

In [5]:
explanation_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are an expert in interpreting and explaining poetry. 
            Your task is to analyze the provided poem and explain how it relates to the programming language 
            it describes. Provide insights into the themes, metaphors, and connections in the poem.
            """,
        ),
        ("human", "Explain this poem:\n{poem}")
    ]
)

explanation_chain = explanation_prompt | chat

final_chain = {"poem": poetry_chain} | explanation_chain
# Example usage
response = final_chain.invoke({"language": "Python"})
print(response)


In the realm of code, a serpent slithers,
Python whispers, in syntax it delivers.
Its elegance dances, with indentation so fine,
Guiding programmers in a language divine.

No need for braces, in this world so clear,
Readability shines, no clutter to fear.
From scripts to apps, its versatility soars,
With libraries vast, for any task it adores.

Dynamic and strong, yet simple and pure,
Python's grace and power, forever endure.
From beginners to pros, it welcomes all,
In its embrace, we stand tall.

With Zen of Python, guiding our way,
Beauty and simplicity in every display.
In the heart of this language, a community thrives,
Together we code, where creativity thrives.

So let us raise our keyboards high,
In praise of Python, reaching the sky.
A language of wonder, a tool so grand,
Python, forever we'll stand.This poem beautifully captures the essence of the Python programming language and its significance in the world of coding. Let's delve into the analysis:

1. **Serpent Symbolism**: 